---
title: "Antibody Performance Dashboard" 
subtitle: "Frankies Diffusion Pipeline"
format:
  dashboard:
    theme: frankies.scss
    icon: body-text
    orientation: columns
    nav-buttons:
      - icon: github
        href: https://github.com/Santollan/Frankies/
---


In [ ]:
#| label: parameters

experiment_dir = "../../experiments/anti_HA_test_CF_02/"
capri_clt_file = "../../experiments/anti_HA_test_CF_02/4_docking/output/10_caprieval/capri_clt.tsv"
capri_ss_file = "../../experiments/anti_HA_test_CF_02/4_docking/output/10_caprieval/capri_ss.tsv"
input_h_json = "../../experiments/anti_HA_test_CF_02/2_diffusion/anti-HA_antibodies_Hchains_aligned.a3m.json"
input_l_json = "../../experiments/anti_HA_test_CF_02/2_diffusion/anti-HA_antibodies_Lchains_aligned.a3m.json"


In [ ]:
#| label: debug-params
#| output: false

# Extract experiment name from path for use elsewhere
import os
experiment_name = os.path.basename(os.path.normpath(experiment_dir))
print(f"Experiment directory: {experiment_dir}")
print(f"CAPRI CLT file: {capri_clt_file}")
print(f"CAPRI SS file: {capri_ss_file}")
print(f"Input H JSON: {input_h_json}")
print(f"Input L JSON: {input_l_json}")

In [ ]:
#| label: load_libs
import pandas as pd
import re, json
import plotly.express as px
import py3Dmol
from IPython.display import display, HTML
from itables import show
import os

In [ ]:
#| label: load_data
clt_df = pd.read_csv(capri_clt_file, sep='\t', comment='#')
ss_df = pd.read_csv(capri_ss_file, sep='\t', comment='#')

with open(input_h_json, 'r') as f:
  h_sequence = json.load(f)["cleaned_sequence"]

with open(input_l_json, 'r') as f:
  l_sequence = json.load(f)["cleaned_sequence"]

## Get best ranking cluster
best_cluster_id = clt_df.loc[clt_df["cluster_rank"] == 1, "cluster_id"].values[0]

## From best cluster, get best model (using VDW)
ss_df_best = ss_df[(ss_df["cluster-id"] == best_cluster_id)]
lowest_vdw_row = ss_df_best.loc[ss_df_best["vdw"].idxmin()]
best_model = lowest_vdw_row["model"].replace('../08_mdscoring/','')

# Get the absolute path of the current working directory
current_dir = os.getcwd()
 

# Navigate up to the experiments directory (2 levels up from postprocess)
experiments_dir = os.path.dirname(os.path.dirname(current_dir))
best_pdb_path = os.path.join(experiments_dir,experiment_name, "4_docking/output/08_mdscoring", best_model)

# Performance

## Column {width=50%}

### Row {height=20%}


In [ ]:
#| label: experiment_display
#| output: asis
import os
experiment_name = os.path.basename(os.path.normpath(experiment_dir))
print(f"<div style='font-size: 20px; font-weight: bold; color:rgb(0, 0, 0);'> {experiment_name}</div>")

In [ ]:
#| label: n_tile
dict(
    # icon = "x-square",
    color = "light",
    value = int(clt_df.loc[clt_df["cluster_rank"] == 1, "n"].values[0])
)

In [ ]:
#| label: score_tile
dict(
    # icon = "x-square",
    color = "light",
  value = f'{float(clt_df.loc[clt_df["cluster_rank"] == 1, "score"].values[0])}\n±{float(clt_df.loc[clt_df["cluster_rank"] == 1, "score_std"].values[0])}'
)

In [ ]:
#| label: irmsd_tile
dict(
    # icon = "x-square",
    color = "light",
    value = f'{float(clt_df.loc[clt_df["cluster_rank"] == 1, "irmsd"].values[0])}\n±{float(clt_df.loc[clt_df["cluster_rank"] == 1, "irmsd_std"].values[0])}'
)

In [ ]:
#| label: dockq_tile
dict(
    # icon = "x-square",
    color = "light",
    value = f'{float(clt_df.loc[clt_df["cluster_rank"] == 1, "dockq"].values[0])}\n±{float(clt_df.loc[clt_df["cluster_rank"] == 1, "dockq_std"].values[0])}'
)

### Row {height=80%}

In [ ]:
#| label: create-graphs
#| output: false
score_fig  = px.box(ss_df, y="score", points="all", labels={"score": "Combined Score"})
score_fig.update_traces(marker_color='#005035', line_color='#005035')
score_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

total_fig  = px.box(ss_df, y="total", points="all", labels={"total": "Total Score"})
total_fig.update_traces(marker_color='#899064', line_color='#899064')
total_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

vdw_fig  = px.box(ss_df, y="vdw", points="all", labels={"vdw": "Van der Waals Energy"})
vdw_fig.update_traces(marker_color='#A49665', line_color='#A49665')
vdw_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

elec_fig  = px.box(ss_df, y="elec", points="all", labels={"elec": "Electrostatic Energy"})
elec_fig.update_traces(marker_color='#101820', line_color='#101820')
elec_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

desolv_fig  = px.box(ss_df, y="desolv", points="all", labels={"desolv": "Desolvation Energy"})
desolv_fig.update_traces(marker_color='#802F2D', line_color='#802F2D')
desolv_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

bsa_fig  = px.box(ss_df, y="bsa", points="all", labels={"bsa": "Buried Surface Area"})
bsa_fig.update_traces(marker_color='#007377', line_color='#007377')
bsa_fig.update_layout(
    showlegend=False,
    paper_bgcolor="white",
    plot_bgcolor="#e0e0e0",
    font_color="#222",
    xaxis=dict(color="#444"),
    yaxis=dict(color="#444")
)

In [ ]:
#| label: show-graphs
#| layout-nrow: 2
#| layout-ncol: 3
score_fig.show()
total_fig.show()
vdw_fig.show()
elec_fig.show()
desolv_fig.show()
bsa_fig.show()

## Column {width=50%}

### Row {height=20%}

In [ ]:
#| label: h_chain
h_seq_html = f"""
<div style='background-color:#f8f9fa; padding:1rem; border-radius:8px; margin-bottom:1rem;'>
  <div style='font-family:monospace; white-space:pre-wrap; word-wrap:break-word;'>
    {h_sequence}
  </div>
</div>
"""

display(HTML(h_seq_html))

In [ ]:
#| label: l_chain
l_seq_html = f"""
<div style='background-color:#f8f9fa; border-radius:8px;'>
  <div style='font-family:monospace; white-space:pre-wrap; word-wrap:break-word;'>
    {l_sequence}
  </div>
</div>
"""

display(HTML(l_seq_html))

### Row {height=80%}

In [ ]:
#| label: make_structure_viewer
#| output: false

with open(best_pdb_path) as pdb_file:
  pdb_data = "".join([x for x in pdb_file])
  
view = py3Dmol.view()
view.addModel(pdb_data)
view.setStyle( {'chain':'A'}, { 'cartoon': {'color': '#005035' }})
view.setStyle( {'chain':'B'}, { 'cartoon': {'color': '#A49665'}})
view.zoomTo()
# view

In [ ]:
#| label: show_structure
print("title=", f"Best-Scoring Complex: {best_model}")

html = f'''
        <html>
        <body style="margin: 0; padding: 0; display: block;">
        {view._make_html()}
        </body>
        </html>
    '''

html = re.sub(r'width: (\S+);', 'width: 100%;', html)
html = re.sub(r'height: (\S+);', 'height: 100vh;', html)

# HTML(pdbview._make_html())
HTML(html)

<!-- 
```{=html}
<script src="https://3Dmol.org/build/3Dmol-min.js"></script>     
<script src="https://3Dmol.org/build/3Dmol.ui-min.js"></script>     
 
<div style="height: 400px; width: 400px; position: relative;" class='viewer_3Dmoljs' data-pdb='2POR' data-backgroundcolor='0xffffff' data-style='stick' data-ui='true'></div>
``` -->

# Data

## Column {width=100%}

### Row {height=50%}

In [ ]:
#| label: model_data
show(ss_df, showIndex = False)

<!-- ### Row {height=50%}

In [ ]:
#| label: cluster_data
show(clt_df, showIndex = False)

-->


# About
## Row
### About FrankIES {orientation="rows" width=100%}
**About**

::: {.card title="FrankIES : Scalable, AI-Based Antibody Design Against 2025 H5N1 Avian Influenza Isolates"}
FrankIES (Frankie Immune Engineering Suite) is a computational pipeline for antibody engineering that integrates multiple advanced tools:

- **Diffusion**: EvoDiff v1.1.0 (Docker container: cford38/evodiff:v1.1.0)
- **Folding**: ESM
- **Docking**: HADDOCK3 (Docker container: cford38/haddock:3)

The pipeline automates the complex process of antibody design, folding, docking, and evaluation, providing comprehensive results visualization through this interactive report.
:::


In [ ]:
#| label: system_info
#| output: asis


import platform
import os
import subprocess
import re
import psutil

# Function to safely run commands
def run_command(command):
    try:
        result = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, text=True)
        return result.strip()
    except subprocess.CalledProcessError as e:
        return f"Command failed: {e.output.strip()}"

# Get OS info
os_info = platform.platform()

# Get CPU info
cpu_info = {}
try:
    with open('/proc/cpuinfo', 'r') as f:
        for line in f:
            if 'model name' in line:
                cpu_info['model'] = re.sub(r'.*model name.*:', '', line).strip()
                break
except:
    cpu_info['model'] = platform.processor()

# Get CPU cores
cpu_info['cores'] = psutil.cpu_count(logical=False)
cpu_info['logical_cores'] = psutil.cpu_count(logical=True)

# Get memory info
memory_gb = round(psutil.virtual_memory().total / (1024**3), 2)

# Get GPU info
gpu_info = {}
nvidia_smi = run_command('nvidia-smi --query-gpu=name,driver_version,memory.total --format=csv,noheader')
if 'failed' not in nvidia_smi and 'not found' not in nvidia_smi.lower():
    gpu_lines = nvidia_smi.strip().split('\n')
    gpu_info['gpus'] = []
    for i, line in enumerate(gpu_lines):
        parts = line.split(', ')
        if len(parts) >= 3:
            gpu_info['gpus'].append({
                'name': parts[0],
                'driver': parts[1],
                'memory': parts[2]
            })

# Get CUDA version - with better error handling
cuda_version = None
try:
    nvcc_output = run_command('nvcc --version')
    if 'not found' not in nvcc_output.lower() and 'failed' not in nvcc_output:
        cuda_match = re.search(r'release (\d+\.\d+)', nvcc_output)
        if cuda_match:
            cuda_version = cuda_match.group(1)
except:
    pass  # Silently handle errors

# Create clean HTML output
output = """
<div class="card mt-3">
  <div class="card-header"><strong>Host System Information</strong></div>
  <div class="card-body">
    <p><strong>Operating System:</strong> {os_info}</p>
    <p><strong>CPU:</strong> {cpu_model}</p>
    <p><strong>Cores:</strong> {cores} physical / {logical_cores} logical</p>
    <p><strong>Memory:</strong> {memory_gb} GB</p>
""".format(
    os_info=os_info,
    cpu_model=cpu_info.get('model', 'Unknown'),
    cores=cpu_info.get('cores', 'Unknown'),
    logical_cores=cpu_info.get('logical_cores', 'Unknown'),
    memory_gb=memory_gb
)

# Add GPU information if available
if gpu_info.get('gpus'):
    output += "<p><strong>GPUs:</strong></p><ul>"
    for gpu in gpu_info['gpus']:
        output += f"<li>{gpu['name']} ({gpu['memory']})</li>"
    output += "</ul>"
    
    if cuda_version:
        output += f"<p><strong>CUDA Version:</strong> {cuda_version}</p>"

output += """
  </div>
</div>
"""

print(output)